In [6]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import math

In [7]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)

In [12]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        a = math.sqrt(3) * math.sqrt(2/m.bias.data.shape[0])
        torch.nn.init._no_grad_uniform_(m.bias.data, -a, a)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        
epochs     = 100
criterion = torch.nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
# Fix magic number
fcn_model = FCN(n_class=34)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

<class 'torch.nn.modules.conv.Conv2d'>
torch.Size([32, 3, 3, 3])
torch.Size([32])


ValueError: Fan in and fan out can not be computed for tensor with fewer than 2 dimensions

In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.to('cuda')# Move your inputs onto the gpu
                labels = Y.to('cuda') # Move your labels onto the gpu
            else:
                inputs, labels = X, Y # Unpack variables into inputs and labels
                
            print('inputs.device', inputs.device)    

            outputs = fcn_model(inputs)
            loss = criterion(outputs, Variable(labels.long()))
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
                
                
            # Checking if early stop conditions met
            if early_stop and i > early_stop_epoch:
                shouldStop = True
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()


def val(epoch):
    
    # set to evaluation mode 
    fcn_model.eval()

    softmax = nn.Softmax(dim = 1)
    
    loss = []
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(val_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
            
        if use_gpu:
            inputs = X.to('cuda')
            labels = Y.to('cuda')
            tar = tar.to('cuda')
        else:
            inputs, labels, tar = X, Y, tar 

        print('inputs.device', inputs.device)    

        outputs = fcn_model(inputs)
        loss.append(criterion(outputs, Variables(tar)))
        prediction = softmax(outputs) 
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(pred, Y))
    
    acc = sum(acc)/len(acc)
    avg_loss = sum(loss)/len(loss) 
    IOU = IOU/iter  
    
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
    return avg_loss, acc, IOU      
       
    
    
    
def test():
    
    softmax = nn.Softmax(dim = 1)
    
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(test_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
        
        if use_gpu:
            inputs = X.to('cuda')
            labels = Y.to('cuda')
            tar = tar.to('cuda')
        else:
            inputs, labels, tar = X, Y, tar 
                
        print('inputs.device', inputs.device)    

        outputs = fcn_model(inputs)  
        print(output.shape)
        prediction = softmax(outputs)
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(prediction, Y))
        
    acc = sum(acc)/len(acc)        
    IOU = IOU/iter

    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
    return acc, IOU
    
if __name__ == "__main__":
    #val(0)  # show the accuracy before training
    train()

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/rileyhadden/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/rileyhadden/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/rileyhadden/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/rileyhadden/Documents/ucsd/wi20/cse253/PA3/Semantic-Segmentation-Pytorch-/dataloader.py", line 100, in __getitem__
    img = np.asarray(Image.open(img_name).convert('RGB'))
  File "/Users/rileyhadden/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py", line 2770, in open
    fp = builtins.open(filename, "rb")
FileNotF